In [7]:
import io
import re
import pandas as pd
from os import walk
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [8]:
#fuction to extract text from PDF file
def extract_text_from_pdf (pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh,
                                        caching=True,
                                        check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

        converter.close()
        fake_file_handle.close()

        if text:
            return text

In [9]:
def invoice_data (pdf_extract):

    #count letter number to ID format
    len_amount = len(str(re.findall(r'\d+', res[31])))
    affiliate = str(invoice_dir[0:4])
    invoice_description = "Monthly coverage"
    invoice_directory = invoice_dir

    if len_amount== 6:
        
        #Invoice amount
        # res[29]= thousands, res[30]= hundreds, res[31]= decimals 
        
        join_amounts = ''.join(re.findall(r'\d+', res[29])) + ''.join(re.findall(r'\d+', res[30])) + "." + ''.join      (re.findall(r'\d+', res[31]))
        invoice_amount = join_amounts

        #Invoice date
        #res[37]= month, res[38]= day, res[39]= year
        join_date = ''.join(re.findall(r'\d+', res[37])) + "/" + ''.join(re.findall(r'\d+', res[38])) + "/" + ''.join(re.findall(r'\d+', res[39]))
        invoice_date = join_date

        #Invoice number
        join_invoice_number = ''.join(re.findall(r'\d+', res[32])) + " " + ''.join(re.findall(r'\d+', res[33])) + " " + ''.join(re.findall(r'\d+', res[34])) + " "          + ''.join(re.findall(r'\d+', res[35])) + " " + ''.join(re.findall(r'\d+', res[36]))
        invoice_number = join_invoice_number
        invoice_number

    else:
        
        #Invoice amount
        # res[29]= hundreds, res[30]= decimals 
        join_amounts = ''.join(re.findall(r'\d+', res[29])) + "." + ''.join(re.findall(r'\d+', res[30]))
        invoice_amount = join_amounts

        #Invoice date
        #res[36]= month, res[37]= day, res[38]= year
        join_date = ''.join(re.findall(r'\d+', res[36])) + "/" + ''.join(re.findall(r'\d+', res[37])) + "/" + ''.join(re.findall(r'\d+', res[38]))
        invoice_date = join_date

        #Invoice number
        join_invoice_number = ''.join(re.findall(r'\d+', res[31])) + " " + ''.join(re.findall(r'\d+', res[32])) + " " + ''.join(re.findall(r'\d+', res[33])) + " "          + ''.join(re.findall(r'\d+', res[34])) + " " + ''.join(re.findall(r'\d+', res[35]))
        invoice_number = join_invoice_number
        invoice_number


    return [invoice_date, invoice_amount, invoice_number, affiliate, invoice_description, invoice_directory]

In [10]:
#path of invoices
mypath = 'C:/zPython/benefits/invoices'

#variable to append invoice name
list_invoices = []
for (dirpath, dirnames, filenames) in walk(mypath):
    list_invoices.extend(filenames)
    break

list_invoices

['ABPI Cigna & Life Disability March 2020.pdf',
 'CMSI_Cigna___Life_Disability_March_2020.pdf',
 'DWTG Cigna & Life Disability March 2020.pdf',
 'MDnA_Cigna___Life_Disability_March_2020.pdf',
 'MHIA_Cigna___Life_Disability_March_2020.pdf',
 'MHPS_Cigna___Life_Disability_March_2020.pdf',
 'MITAC-A_Cigna___Life_Disability_March_2020.pdf',
 'MNES_Cigna___Life_Disability_March_2020.pdf',
 'Mohawk_Cigna___Life_Disability_March_2020.pdf',
 'MSSA_Cigna___Life_Disability_March_2020.pdf',
 'PWPS_Cigna___Life_Disability_March_2020.pdf']

In [11]:
#loop
invoice_information = []

for x in list_invoices:
    
    invoice_dir = x
    invoice_full_dir = mypath + "/" + invoice_dir
    invoice_text = extract_text_from_pdf(invoice_full_dir)

    # to extract words from string to list 
    res = re.findall(r'\w+', invoice_text)

    #call fuction to extract invoice information 'date', 'amount', 'invoice number'
    newInfo = invoice_data(res)
    invoice_information.append(newInfo)

In [12]:
df = pd.DataFrame(invoice_information)
df.rename( columns={0:'invoice_date',1:'Amount',2:'invoice_num',3:'Affiliate',4:'Description',5:'attachment'}, inplace=True )
df

,invoice_date,Amount,invoice_num,Affiliate,Description,attachment
0,03/05/2020,1911.40,0813608934 0002 030120 20065 1,ABPI,Monthly coverage,ABPI Cigna & Life Disability March 2020.pdf
1,03/01/2020,9751.72,0813608934 0003 030120 20061 4,CMSI,Monthly coverage,CMSI_Cigna___Life_Disability_March_2020.pdf
2,03/01/2020,8249.16,0813608934 0008 030120 20061 6,DWTG,Monthly coverage,DWTG Cigna & Life Disability March 2020.pdf
3,03/01/2020,44063.93,0813608934 0023 030120 20061 0,MDnA,Monthly coverage,MDnA_Cigna___Life_Disability_March_2020.pdf
4,03/05/2020,16068.08,0813608934 0001 030120 20065 4,MHIA,Monthly coverage,MHIA_Cigna___Life_Disability_March_2020.pdf
5,03/01/2020,115526.38,0813608934 0021 030120 20061 7,MHPS,Monthly coverage,MHPS_Cigna___Life_Disability_March_2020.pdf
6,03/01/2020,31633.97,0813608934 0007 030120 20061 0,MITA,Monthly coverage,MITAC-A_Cigna___Life_Disability_March_2020.pdf
7,03/01/2020,337.99,0813608934 0004 030120 20061 1,MNES,Monthly coverage,MNES_Cigna___Life_Disability_March_2020.pdf
8,03/01/2020,1061.25,0813608934 0024 030120 20061 6,Moha,Monthly coverage,Mohawk_Cigna___Life_Disability_March_2020.pdf
9,03/01/2020,14013.49,0813608934 0000 030120 20061 5,MSSA,Monthly coverage,MSSA_Cigna___Life_Disability_March_2020.pdf


In [14]:
export_path = 'C:/zPython/benefits/clean_data/invoices_information.csv'
df.to_csv(export_path, index=False)